In [6]:
from dataclasses import dataclass
import math

@dataclass(frozen=True)
class Vector3:
    x: float
    y: float
    z: float

    # Vector addition
    def __add__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    # Vector subtraction
    def __sub__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )

    # Scalar multiplication
    def __mul__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x * scalar,
            self.y * scalar,
            self.z * scalar
        )

    __rmul__ = __mul__

    # Dot product
    def dot(self, other: "Vector3") -> float:
        return (
            self.x * other.x +
            self.y * other.y +
            self.z * other.z
        )

    # Cross product
    def cross(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )

    # Magnitude (length)
    def magnitude(self) -> float:
        return math.sqrt(self.dot(self))

    # Unit vector
    def unit_vector(self) -> "Vector3":
        mag = self.magnitude()
        if mag == 0:
            raise ValueError("Cannot normalize a zero vector")
        return self * (1 / mag)

In [ ]:
# Semi Major Axis (a)
def compute_sma(mu:float, energy:float):
    return -mu/2*energy


In [ ]:
# Eccentricity (e)
import math

def compute_ecc(energy: float, mu: float, h:float):
    return math.sqrt(1+(2*math.pow(h, 2)*energy)/(math.pow(mu, 2)))

In [ ]:
# Inclination (i)
import math

def compute_inc(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector
    h_ = h_.unit_vector
    return math.acos(x=Z_.dot(h_))


In [ ]:
# RAAN (capital omega) Ω
import math

def compute_raan(Nx: float, Ny: float):
    return math.atan2(y=Ny, x=Nx)

In [ ]:
# Argument of Perigeee (omega sub p) ωp
import math

def compute_argp(h_: Vector3, N_: Vector3, B_: Vector3):
    h_ = h_.unit_vector
    N_ = N_.unit_vector
    B_ = B_.unit_vector
    return math.atan2(y=h_.dot(N_.cross(B_)), x=N_.dot(B_))



In [ ]:
# True Anomaly (nu) ν
import math

def compute_ta(v_: Vector3, r_: Vector3, B_: Vector3):
    cosTa = r_.dot(B_)/(r_.magnitude*B_.magnitude)
    ta = math.acos(cosTa)
    if r_.dot(v_) < 0:
        ta = 2*math.pi - ta
    return ta

In [ ]:
# Period (TP)
import math

def compute_period(a: float, mu: float):
    return 2*math.pi*math.sqrt(math.pow(a, 3)/mu)

In [ ]:
# Apogee (r sub a) ra
